In [1]:
import numpy as np
import pandas as pd

from keras import Model
from keras.layers import Input, LSTM, Dense

Using TensorFlow backend.


In [2]:
# set constants
BATCH_SIZE = 64
EPOCHS = 100
LATENT_DIM = 256
NUM_SAMPLES = 10000
DATA_PATH = 'encoded.csv'
ENCODING = 'utf-8'

In [3]:
# create variables
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [4]:
# load data file
array = pd.read_csv(DATA_PATH).to_numpy()
for index, row in enumerate(array):
    if (index >= NUM_SAMPLES):
        break
    input_text = row[1]
    target_text = row[2]
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)

    for char in input_text:
        if (char not in input_characters):
            input_characters.add(char)
    for char in target_text:
        if (char not in target_characters):
            target_characters.add(char)

In [5]:
# sort datalists
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [6]:
# set variables
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 72
Max sequence length for inputs: 20
Max sequence length for outputs: 22


In [8]:
# get token indexes
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [9]:
# prepare data for training
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if (t > 0):
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[chr(ord(' ')+1)]] = 1.
    decoder_target_data[i, t:, target_token_index[chr(ord(' ')+1)]] = 1.

In [11]:
# prepare input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(LATENT_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [12]:
# prepare decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(LATENT_DIM, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [13]:
# create model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [14]:
# train model
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, 
          batch_size=BATCH_SIZE, 
          epochs=EPOCHS, 
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 14s 2ms/step - loss: 2.2960 - val_loss: 2.7327
Epoch 2/100
8000/8000 [==============================] - 15s 2ms/step - loss: 1.8356 - val_loss: 2.1830
Epoch 3/100
8000/8000 [==============================] - 14s 2ms/step - loss: 1.5372 - val_loss: 1.8988
Epoch 4/100
8000/8000 [==============================] - 14s 2ms/step - loss: 1.3682 - val_loss: 1.8162
Epoch 5/100
8000/8000 [==============================] - 14s 2ms/step - loss: 1.2380 - val_loss: 1.6196
Epoch 6/100
8000/8000 [==============================] - 14s 2ms/step - loss: 1.1328 - val_loss: 1.6079
Epoch 7/100
8000/8000 [==============================] - 14s 2ms/step - loss: 1.0555 - val_loss: 1.4284
Epoch 8/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.9876 - val_loss: 1.3716
Epoch 9/100
8000/8000 [==============================] - 15s 2ms/step - loss: 0.9276 - val_loss: 1.3162
Epoch 10/100
800

8000/8000 [==============================] - 16s 2ms/step - loss: 0.0051 - val_loss: 0.6345
Epoch 79/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.0050 - val_loss: 0.6219
Epoch 80/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.0042 - val_loss: 0.6227
Epoch 81/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.0043 - val_loss: 0.6176
Epoch 82/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.0046 - val_loss: 0.6271
Epoch 83/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.0047 - val_loss: 0.6266
Epoch 84/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.0045 - val_loss: 0.6358
Epoch 85/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.0046 - val_loss: 0.6087
Epoch 86/100
8000/8000 [==============================] - 16s 2ms/step - loss: 0.0043 - val_loss: 0.5893
Epoch 87/100
8000/8000 [==============================] - 16s 2ms/st

In [15]:
model.save('encryption-model.h5')

In [16]:
# build sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(LATENT_DIM,))
decoder_state_input_c = Input(shape=(LATENT_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [17]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [18]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]
    return decoded_sentence

In [20]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print(f"Input sentence: {input_texts[seq_index]}")
    print(f"Encoded sentence: {decoded_sentence}")

-
Input sentence: Hi.
Encoded sentence: Ij/

-
Input sentence: Hi.
Encoded sentence: Ij/

-
Input sentence: Hi.
Encoded sentence: Ij/

-
Input sentence: Run!
Encoded sentence: Svo"

-
Input sentence: Who?
Encoded sentence: Xip@

-
Input sentence: Wow!
Encoded sentence: Xpx"

-
Input sentence: Fire!
Encoded sentence: Gjsf"

-
Input sentence: Fire!
Encoded sentence: Gjsf"

-
Input sentence: Help!
Encoded sentence: Ifmq"

-
Input sentence: Jump.
Encoded sentence: Kvnq/

-
Input sentence: Stop!
Encoded sentence: Tupq"

-
Input sentence: Stop!
Encoded sentence: Tupq"

-
Input sentence: Wait.
Encoded sentence: Xbju/

-
Input sentence: Hello!
Encoded sentence: Ifmmp"

-
Input sentence: Hello!
Encoded sentence: Ifmmp"

-
Input sentence: Hello!
Encoded sentence: Ifmmp"

-
Input sentence: I try.
Encoded sentence: J!usz/

-
Input sentence: I won!
Encoded sentence: J!xpo"

-
Input sentence: I won.
Encoded sentence: J!xpo/

-
Input sentence: Oh no!
Encoded sentence: Pi!op"

-
Input sentence: Relax.